## Install the repo

In [ ]:
!git clone --recursive https://github.com/Digre01/Visual-Place-Recognition-Project.git

In [ ]:
%cd Visual-Place-Recognition-Project/image-matching-models
!pip install -e .[all]

In [ ]:
!pip install faiss-cpu

## Download datasets

In [ ]:
!python download_datasets.py

## Pipeline

In [ ]:
import os
import shutil

CONFIGS = [
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "loftr"},
    {"vpr": "cosplace", "backbone": "ResNet18", "dim": 512, "matcher": "superpoint-lg"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "loftr"},
    {"vpr": "netvlad", "backbone": "VGG16", "dim": 4096, "matcher": "superpoint-lg"}
]

for conf in CONFIGS:
    v_name = conf["vpr"]
    m_name = conf["matcher"]
    log_tag = f"train_{v_name}"
    
    print(f"\n{'='*60}\Running: {v_name.upper()} + {m_name.upper()}\n{'='*60}")
    
    #RETRIVAL
    !python VPR-methods-evaluation/main.py \
        --method={v_name} --backbone={conf['backbone']} --descriptors_dimension={conf['dim']} \
        --num_workers 4 --batch_size 32 \
        --log_dir {log_tag} \
        --image_size 512 512 \
        --database_folder 'data/svox/images/test/gallery' \
        --queries_folder 'data/svox/images/test/queries_sun' \
        --recall_values 1 5 10 20 --num_preds_to_save 20
    

    base_log = f"logs/{log_tag}"
    latest_run = sorted(os.listdir(base_log))[-1]
    preds_dir = os.path.join(base_log, latest_run, "predictions")
    

    #MATCHING
    out_inliers = f"inliers_full_{v_name}_{m_name}"
    !python match_queries_preds.py \
        --preds-dir {preds_dir} \
        --out-dir {out_inliers} \
        --matcher {m_name} \
        --num-preds 20
    
    #TRAINING
    csv_file = f"dataset_{v_name}_{m_name}.csv"
    model_file = f"model_{v_name}_{m_name}.joblib"
    
    !python prepare_logistic_data.py --preds-dir {preds_dir} --inliers-dir {out_inliers} --output-csv {csv_file}
    !python train_logistic_model.py --csv-path {csv_file} --model-out {model_file}

## Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
import glob

plt.figure(figsize=(12, 7))

x_test = np.linspace(0, 100, 500).reshape(-1, 1)

colors = ['blue', 'cyan', 'red', 'orange']
styles = ['-', '--', '-', '--']

for i, conf in enumerate(CONFIGS):
    m_path = f"model_{conf['vpr']}_{conf['matcher']}.joblib"
    if os.path.exists(m_path):
        model = joblib.load(m_path)
        # Calcola la probabilità P(y=1 | x)
        probs = model.predict_proba(x_test)[:, 1]
        
        label = f"{conf['vpr']} + {conf['matcher']}"
        plt.plot(x_test, probs, label=label, color=colors[i], linestyle=styles[i], lw=2)

plt.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
plt.title("Probability correct match", fontsize=14)
plt.xlabel("Num inliers (Top-1)", fontsize=12)
plt.ylabel("Probability (distance < 25m)", fontsize=12)
plt.grid(True, which='both', linestyle='--', alpha=0.4)
plt.legend(loc='lower right', fontsize=10)
plt.xlim(0, 100) 
plt.ylim(0, 1.05)

plt.savefig("logistic_comparison.png")
plt.show()